In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

The purpose of this file is to read and extract language data from CIA website.

Output: a csv file with 2 columns - country and language (uncleaned) 

In [3]:
URL = 'https://www.cia.gov/library/publications/the-world-factbook/fields/402.html'
page = requests.get(URL)

#soup = BeautifulSoup(page.content, 'html.parser')

In [4]:
 class HTMLTableParser:
       
        def parse_url(self, url):
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'lxml')
            return [(table['id'],self.parse_html_table(table))\
                    for table in soup.find_all('table')]  
    
        def parse_html_table(self, table):
            n_columns = 0
            n_rows=0
            column_names = []
    
            # Find number of rows and columns
            # we also find the column titles if we can
            for row in table.find_all('tr'):
                
                # Determine the number of rows in the table
                td_tags = row.find_all('td')
                if len(td_tags) > 0:
                    n_rows+=1
                    if n_columns == 0:
                        # Set the number of columns for our table
                        n_columns = len(td_tags)
                        
                # Handle column names if we find them
                th_tags = row.find_all('th') 
                if len(th_tags) > 0 and len(column_names) == 0:
                    for th in th_tags:
                        column_names.append(th.get_text())
    
            # Safeguard on Column Titles
            if len(column_names) > 0 and len(column_names) != n_columns:
                raise Exception("Column titles do not match the number of columns")
    
            columns = column_names if len(column_names) > 0 else range(0,n_columns)
            df = pd.DataFrame(columns = columns,
                              index= range(0,n_rows))
            row_marker = 0
            for row in table.find_all('tr'):
                column_marker = 0
                columns = row.find_all('td')
                for column in columns:
                    df.iat[row_marker,column_marker] = column.get_text()
                    column_marker += 1
                if len(columns) > 0:
                    row_marker += 1
                    
            # Convert to float if possible
            for col in df:
                try:
                    df[col] = df[col].astype(float)
                except ValueError:
                    pass
            
            return df

In [5]:
hp = HTMLTableParser()
table = hp.parse_url(URL)[0][1] # Grabbing the table from the tuple
table.head()

,Country,Languages
0,\nAfghanistan\n,\n\n\n \n Afghan Persian or Da...
1,\nAkrotiri\n,"\n\n\n \n English, Greek\n ..."
2,\nAlbania\n,\n\n\n \n Albanian 98.8% (offi...
3,\nAlgeria\n,"\n\n\n \n Arabic (official), ..."
4,\nAmerican Samoa\n,\n\n\n \n Samoan 88.6% (closel...


In [6]:
table.Country = table.Country.apply(lambda x: x.strip('\n'))
table.Languages = table.Languages.apply(lambda x: x.strip('\n'))

In [7]:
table

,Country,Languages
0,Afghanistan,\n Afghan Persian or Dari (of...
1,Akrotiri,"\n English, Greek\n \n..."
2,Albania,\n Albanian 98.8% (official -...
3,Algeria,"\n Arabic (official), French..."
4,American Samoa,\n Samoan 88.6% (closely rela...
...,...,...
236,Western Sahara,"\n Standard Arabic, Hassaniya..."
237,World,"most-spoken language:Â English 16.5%, Mandarin..."
238,Yemen,\n Arabic (official)\n ...
239,Zambia,"\n Bemba 33.4%, Nyanja 14.7%,..."


In [8]:
# drop world and EU rows
table = table.drop(table[table.Country.isin(['World', 'European Union'])].index)

In [9]:
table.to_csv('../data/language.csv', index = False)